# Task 1

The first task tests your Python skills. You need to develop a simple game consisting of a rectangular grid (of size height x width ) where each cell has a random integer value between 0 and 9. An agent starts at the upper-left corner of the grid and must reach the lower-right corner of the grid as fast as possible.

In [1]:
import numpy as np
import random
from abc import ABC, abstractmethod

In [2]:
class Game():
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.maze = (np.random.rand(self.height, self.width) * 10).astype('int')
        self.start = self.maze[0][0]
        self.end = self.maze[self.height - 1][self.width - 1]
        self.game_complete = False
        # Pointer
        self.pointer_x = 0
        self.pointer_y = 0
        self.pointer_val = self.maze[0][0]
        self.pointer = f"Currently at {self.start}, located on x={0}, y={0}"
        # Records
        self.steps_taken = 0
        self.time_spent = 0
    
    def move_pointer(self, x_pos, y_pos):
        # Check if x_pos and y_pos are legal (step size of 1 unit, that does not exceed maze boundary)
        if(abs(x_pos - self.pointer_x) > 1 or (x_pos >= self.width or x_pos < 0)):
            print("X Step too large")
            return
        if(abs(y_pos - self.pointer_y) > 1 or (y_pos >= self.height or y_pos < 0)):
            print("Y Step too large")
            return
        # Check that step is not diagonal (i.e. it is not the case that both x and y positions have changed)
        if((x_pos != self.pointer_x) and (y_pos != self.pointer_y)):
            print("Cannot move diagonally!")
            return
        # If all is legal, make the step
        # Update records
        self.steps_taken += 1
        self.time_spent += self.pointer_val
        # Move pointer
        self.pointer_x = x_pos
        self.pointer_y = y_pos
        self.pointer_val = self.maze[y_pos][x_pos]
        self.pointer = f"Currently at {self.pointer_val}, located on x={x_pos}, y={y_pos}"
        print(f"{self.pointer}")
        # Check if after the move, the agent has won the game.
        if(self.pointer_x == (self.width - 1) and self.pointer_y == (self.height - 1)):
            self.game_complete = True
            print("Won the game! 🥳")
    
    def reset(self):
        print("Reseting game")
        self.pointer_x = 0
        self.pointer_y = 0
        self.steps_taken = 0
        self.time_spent = 0
        self.pointer = f"Currently at {self.start}, located on x={0}, y={0}"

In [3]:
#Test game
game1 = Game(width=5, height=5)
print(game1.maze)
game1.move_pointer(x_pos=1, y_pos=0)

[[9 9 8 8 0]
 [9 4 3 0 6]
 [6 7 4 0 5]
 [5 7 7 2 8]
 [8 8 9 8 0]]
Currently at 9, located on x=1, y=0


## Random Agent

Our baseline agent needs to be superior to a random agent in reaching the end of the game. We implement a random agent to test their performance for comparison's sake.

In [4]:
class Agent(ABC):
    @abstractmethod
    def __init__(self, game: Game):
        self.game = game
        
    @abstractmethod
    def play_game(self):
        pass

In [5]:
class Random_Agent(Agent):
    def __init__(self, game: Game):
        self.game = game
    
    def play_game(self):
        # While the game is not complete...
        while(not self.game.game_complete):
            # Determine if we're moving along the y axis (if not, the step is along the x axis)
            move_y = random.choice((0, 1))
            # Determine if our direction is negative or positive (e.g. backwards along the y axis vs forwards along the y axis)
            direction = random.choice((-1, 1))

            # If the move is along the x axis, ensure the move does not exceed the boundaries of the maze (in that case, go back to start of loop)
            # Then, move in the random direction alongside the x axis
            if(move_y == 0):
                new_x = self.game.pointer_x + direction
                new_y = self.game.pointer_y
                if(new_x >= self.game.width or new_x < 0):
                    continue
                print("Moving x")
                self.game.move_pointer(x_pos=new_x, y_pos=new_y)
            #Do the same if it were determined that the move shoudl be along the y axis...
            else:
                new_x = self.game.pointer_x
                new_y = self.game.pointer_y + direction
                if(new_y >= self.game.height or new_y < 0):
                    continue
                print("Moving y")
                self.game.move_pointer(x_pos=new_x, y_pos=new_y)
        return


In [9]:
game2 = Game(width=40, height=20)
random_agent = Random_Agent(game2)
random_agent.play_game()
print(f"\nTime spent: {game2.time_spent}, Steps taken: {game2.steps_taken}")

ly at 8, located on x=14, y=16
Moving y
Currently at 0, located on x=14, y=17
Moving y
Currently at 8, located on x=14, y=18
Moving y
Currently at 0, located on x=14, y=17
Moving y
Currently at 8, located on x=14, y=18
Moving x
Currently at 2, located on x=15, y=18
Moving x
Currently at 8, located on x=14, y=18
Moving x
Currently at 2, located on x=15, y=18
Moving x
Currently at 2, located on x=16, y=18
Moving y
Currently at 9, located on x=16, y=19
Moving y
Currently at 2, located on x=16, y=18
Moving y
Currently at 1, located on x=16, y=17
Moving y
Currently at 2, located on x=16, y=18
Moving y
Currently at 9, located on x=16, y=19
Moving x
Currently at 9, located on x=17, y=19
Moving x
Currently at 1, located on x=18, y=19
Moving x
Currently at 9, located on x=17, y=19
Moving x
Currently at 1, located on x=18, y=19
Moving x
Currently at 6, located on x=19, y=19
Moving y
Currently at 9, located on x=19, y=18
Moving y
Currently at 6, located on x=19, y=17
Moving x
Currently at 6, loca

# Baseline Agent

Our baseline agent will move diagonally towards the bottom right. Once it reaches an edge, it then moves (either downwards or across to the left, depending on maze dimensions), until it reaches the end point

In [10]:
class Baseline_Agent(Agent):
    def __init__(self, game: Game):
        self.game = game
    
    def play_game(self):
        while(not self.game.game_complete):
            # If we reached the bottom edge, move across to the right
            if (self.game.pointer_y == self.game.height - 1):
                new_x = self.game.pointer_x + 1
                new_y = self.game.pointer_y
                self.game.move_pointer(x_pos=(new_x), y_pos=(new_y))
            # Else if we reached the right edge, move downwards
            elif (self.game.pointer_x == self.game.width - 1):
                new_x = self.game.pointer_x
                new_y = self.game.pointer_y + 1
                self.game.move_pointer(x_pos=(new_x), y_pos=(new_y))
            # Else, move diagonally, by going through two steps
            else:
                new_x_1 = self.game.pointer_x + 1
                new_y_1 = self.game.pointer_y
                self.game.move_pointer(x_pos=(new_x_1), y_pos=(new_y_1))
                new_x_2 = self.game.pointer_x
                new_y_2 = self.game.pointer_y + 1
                self.game.move_pointer(x_pos=(new_x_2), y_pos=(new_y_2))

        return
            

In [16]:
game3 = Game(width=40, height=20)
print(game3)
baseline_agent = Baseline_Agent(game3)
baseline_agent.play_game()
print(f"\nTime spent: {game3.time_spent}, Steps taken: {game3.steps_taken}")

Currently at 0, located on x=1, y=0
Currently at 7, located on x=1, y=1
Currently at 6, located on x=2, y=1
Currently at 6, located on x=2, y=2
Currently at 9, located on x=3, y=2
Currently at 4, located on x=3, y=3
Currently at 0, located on x=4, y=3
Currently at 9, located on x=4, y=4
Currently at 2, located on x=5, y=4
Currently at 8, located on x=5, y=5
Currently at 4, located on x=6, y=5
Currently at 2, located on x=6, y=6
Currently at 9, located on x=7, y=6
Currently at 8, located on x=7, y=7
Currently at 0, located on x=8, y=7
Currently at 5, located on x=8, y=8
Currently at 5, located on x=9, y=8
Currently at 8, located on x=9, y=9
Currently at 6, located on x=10, y=9
Currently at 0, located on x=10, y=10
Currently at 2, located on x=11, y=10
Currently at 5, located on x=11, y=11
Currently at 7, located on x=12, y=11
Currently at 2, located on x=12, y=12
Currently at 7, located on x=13, y=12
Currently at 6, located on x=13, y=13
Currently at 1, located on x=14, y=13
Currently a